In [ ]:
%run TreePrinter.ipynb

In [ ]:
import findspark
findspark.init('/usr/local/spark')

In [ ]:
import pyspark,pyodbc

In [ ]:
from pyspark import SparkContext, SparkConf, SQLContext
import _mssql
import pandas as pd

## data from mysql

In [ ]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.config("spark.jars", "/home/mosaico/Documents/mosaico/sqljdbc_8.2/mysql-connector-java-8.0.22.jar") \
    .master("local").appName("PySpark_MySQL_test").getOrCreate()


studenti = spark_session.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/mosaico") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "INFORMATION_SCHEMA.TABLES") \
    .option("user", "PanDario").option("password", "Mosaico2020").load()
studenti.show(vertical=True)

## Data from json

In [29]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.appName("DataframeFromJson").getOrCreate()

spark_session.sql("CREATE TABLE STUDENTI (NOME varchar(255),COGNOME varchar(255),CF varchar(255),IBAN varchar(255))")
df.printSchema()

AnalysisException: "Hive support is required to CREATE Hive TABLE (AS SELECT);;\n'CreateTable `STUDENTI`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists\n"

In [ ]:
def listTables(res:pd.core.frame.DataFrame):
    obj=res['TABLE_NAME']
    list=[]
    for x in obj:
        list.append(x)
    return list
def listCols(res:pd.core.frame.DataFrame):
    obj=res.columns
    list=[]
    for c in obj:
        list.append(c)
    return list

QUERY DI TEST

In [ ]:
def treeGen(conn,dbName):
    #TODO fix this! u lazy!
    conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={ser};DATABASE={database};UID={user};PWD={password}')
    query="SELECT * FROM  INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'"
    res=pd.read_sql(query, conn)
    tablesName=listTables(res)
    tables=[]
    root=Node(dbName,'DB')
    for t in tablesName:
        tables.append(Node(t,'table'))
    root.add_child(tables)
    for t in tables:
        query="SELECT * FROM  {0} ".format(t.data)
        res=pd.read_sql(query, conn)
        leafs=listCols(res)
        for l in leafs:
            n=Node(l,'column')
            t.add_child(n)
    #pprint_tree(root)
    return root

In [ ]:
### TEST DATAFRAME

In [ ]:
sparkDF =  spark.createDataFrame(res)

### Cali for ODRL

In [ ]:
import pycali

In [ ]:
from pycali.vocabulary import ODRLVocabulary
odrl = ODRLVocabulary()

#odrl.actions

### AD HOC USER
CREATE USER Tester WITH PASSWORD = 'SuperSecure.1'; <br>
SELECT * FROM sys.database_principals WHERE authentication_type_desc='DATABASE'

### SQL QUERY
sql query has this structure
<pre>
SELECT something FROM some table
   WHERE some conditions are satisfied

UPDATE some table SET something
   WHERE some conditions are satisfied
<code>

### ODRL POLICY TEST

In [ ]:
import json,re
data={}
with open("policy.json") as policy:
    data=json.load(policy)
print(json.dumps(data, indent=2))

In [ ]:
MAINSERVER="mosaico.database.windows.net/Test"

def jsonPermissionSearch(data,assignee,targetColumn,targetTable):
    for e in data['permission']:
        #if rule that define access to the whole table
        if e['assignee']=="{0}:user:{1}".format(MAINSERVER,assignee) and e['target']=="{0}:{1}".format(MAINSERVER,targetTable):
            #print("rule for the whole table found")
            return e
        #rule specific to table:column
        elif e['assignee']=="{0}:user:{1}".format(MAINSERVER,assignee) and e['target']=="{0}:{1}:{2}".format(MAINSERVER,targetTable,targetColumn) and targetColumn!='*':
            #print("rule for the column found")
            return e
        else:
            pass
    #print('no rule found')
    return None

In [ ]:
query="SELECT * FROM STUDENTI ;SELECT NOME,COGNOME FROM DOCENTI;"
query=query.split(';')
#multiple query split-check for select, not for updates
completeQueryList=[]
for q in query:
    if len(q.strip())>0:
        res=re.split('SELECT|FROM',q)
        res.remove('')
        res=[r.strip() for r in res]
        #res[0] -> select args
        #res[1] -> from args
        for column in res[0].split(','):
            policy=jsonPermissionSearch(data,'Tester',column,res[1])
            if policy and policy['action']=='read':
                completeQueryList.append('SELECT {0} FROM {1}'.format(column,res[1]))
print(';'.join(completeQueryList))


#### TODO
Aggregare query appartenenti alla stessa tabella per evitare query eccessive.  
Se devo selezionare ad esempio nella tabella ALPHA le tuple con NOME==a , NOME==b , NOME==c  deve risultare in :  
    SELECT * FROM ALPHA WHERE NAME in ('a', 'b' , 'c')

# Json Data sample

In [ ]:
jsonData='''
[{
  "@type": "Agreement",
  "uid": "mosaico.database.windows.net/Test:policy:1",
  "permission": [{
      "target": "mosaico.database.windows.net/Test:STUDENTI",
      "assignee": "mosaico.database.windows.net/Test:user:Tester",
      "action": "read"
  }]
  },
  {
  "@type": "Agreement",
  "uid": "mosaico.database.windows.net/Test:policy:2",
  "permission": [
    {
      "target": "mosaico.database.windows.net/Test:DOCENTI:NOME",
      "assignee": "mosaico.database.windows.net/Test:user:Tester",
      "action": "read"
    }
  ]
}]
'''

# Json parser ad hoc
TODO: prevedere gruppi di usr per gli accessi

In [ ]:
#!pip3 install parse
import json,pprint
from parse import *

def parseIRIdb(iri):
    std_target="{serverUrl}/{db}:{table}:{name}"
    p=parse(std_target,iri)
    if p==None:
        std_target="{serverUrl}/{db}:{table}"
        p=parse(std_target,iri)
        if p==None:
            std_target="{serverUrl}/{db}"
            p=parse(std_target,iri)
    print("\turl: \t\t",p["serverUrl"])
    print("\tdb: \t\t",p["db"])
    if "table" in p:
        print("\ttable: \t\t",p["table"])
    if "name" in p:
        print("\tname: \t\t",p["name"])
        
def parseIRIusr(iri):
    std_target="{serverUrl}/{db}:{table}:{name}"
    p=parse(std_target,iri)
    print("\tname: \t\t",p["name"])
    
def parseODLR(jsonData):
    doc = json.loads(jsonData)
    for rule in doc:
        rule_dict=rule.get("permission")[0]
        rule_type=rule.get("@type")
        rule_id=rule.get("uid")
        print(rule_id)
        trg=rule_dict.get("target")
        parseIRIdb(trg)
        ass=rule_dict.get("assignee")
        parseIRIusr(ass)
        print("\tpermission: \t",rule_dict.get("action"))


parseODLR(jsonData)
    

## RDFLIB time

In [ ]:
#!pip3 install rdflib-jsonld
from rdflib import Graph, plugin
import rdflib
g=Graph().parse(data=jsonData,format='json-ld')
g.serialize(format='nt')

In [ ]:
import pprint
okString='''{
    "@id": "http://example.org/about",
    "http://purl.org/dc/terms/title": [
        {
            "@type": "literal",
            "@language": "en",
            "@value": "Someone's Homepage"
        }
    ],
    "@authorization":"read"
}'''

print(isinstance(okString,dict))
g=Graph().parse(data=okString,format='json-ld')
list(g)

# TREE GENERATOR

In [ ]:
tree1=treeGen(conn,"Test")

In [ ]:
def authFinder(subject,file,permType='read'):
    doc = json.loads(file)
    targets=[]
    if permType=='read' or permType=='write': #correct permission
        rules=[]
        for rule in doc:
            rule_dict=rule.get("permission")[0]
            rule_type=rule.get("@type")
            ass=rule_dict.get("assignee")
            if rule_type==permType and ass==subject:
                trg=rule_dict.get("target")
                parseURIdb(trg)
                targets.append(trg)
        return rules
    else:
        return false
def URIgenerator(Node,server):
    rootReached=False
    uri=""
    while not rootReached:
        if uri!="":
            uri=Node.data+":"+uri
        else:
            uri=Node.data
        if Node.parent!= Node:
            Node=Node.parent
        else:
            rootReached=True
    return server+"/"+uri

def treeAnalyzer(subject,authFile,server,request='read'):
    global conn,tree1
    auths=authFinder(subject,authFile)
    tbaNodes=[]
    tbaNodes.append(tree1)
    #tbaNodes.append(treeGen(conn,"Test")) #SERVER NAME STATICO!
    while True:
        for node in tbaNodes:
            if len(node.children)>0:
                for child in node.children:
                    x=URIgenerator(child,server)
                    print(x)
                    tbaNodes.append(child)
            tbaNodes.remove(node)
        if len(tbaNodes)==0:
            break
        
    else:
        print("no child nodes")
    #pprint_tree(treeRoot)

In [ ]:
serv="mosaico.database.windows.net"
#pprint_tree(treeGen(conn,"Test"))
treeAnalyzer("Tom",jsonData,serv)
